# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [ ]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import numpy as np

# Import API key
from api_keys import geoapify_key

In [ ]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [ ]:
# Configure the map plot
map_plot_1 = city_data_df.hvplot.points('Lng', "Lat", geo = True, tiles = "OSM", size="Humidity", scale=0.5, color="City")

# Display the map
map_plot_1

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [ ]:
# Narrow down cities that fit criteria and drop any results with null values
# Drop any rows with null values
# Display sample data
reduced_list = city_data_df[(city_data_df['Max Temp']>23.5) & (city_data_df['Max Temp']<27.5) & 
    (city_data_df['Cloudiness']<4) & (city_data_df['Wind Speed']<4)]
drop_na = reduced_list.dropna()
reduced_list

### Step 3: Create a new DataFrame called `hotel_df`.

In [ ]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# Display sample data
hotel_df = reduced_list[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()
hotel_df["Hotel Name"] = ''
hotel_df

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [ ]:
# Set parameters to search for a hotel
radius = 10000
latitude = []
longitude = []
params = {'apiKey':geoapify_key, 'categories': "accommodation.hotel"}

# Print a message to follow up the hotel search
print("Starting hotel search")

for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    filters = [f'circle:{longitude},{latitude},{radius}']
    bias = [f'proximity:{longitude},{latitude}']
    params['categories'] = f'accommodation.hotel'
    params['filter'] = filters
    params['bias'] = bias
        
# Set base URL
    base_url = "https://api.geoapify.com/v2/places"

# Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params).json()
    #data = name_address.json()

# Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
# Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
hotel_df.dtypes

In [ ]:
%%capture --no-display
#%%capture --no-display
#This is in 09-live banking from 10/7/2024 class
# Configure the map plot
city_hotel_name_map = hotel_df.hvplot.points(
    'Lng', 
    'Lat', 
    geo = True,
    tiles = 'OSM', 
    size = 'Humidity', 
    scale = 1,
    color = 'Hotel Name',
    hover_cols = ['Hotel Name', 'Country'],
    frame_width = 800, 
    frame_height = 600
)

# Display the map
city_hotel_name_map